In [170]:
import polars as pl
import datasets as ds
import json


out_file = '../datasets/compiled.jsonl'

In [166]:
dataset = ds.load_dataset("OpenAssistant/oasst1", split="train[:-1]")

df = pl.from_arrow(dataset.data.table).filter(
    pl.col('lang') == 'en'
)

Found cached dataset parquet (/home/lawrence/.cache/huggingface/datasets/OpenAssistant___parquet/OpenAssistant--oasst1-2960c57d7e52ab15/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [167]:
p_df = df.filter(
    pl.col('parent_id') == None
).select(['message_id', 'parent_id', 'text', 'role'])

r_df = df.filter(
    (pl.col('parent_id') != None) 
    & (pl.col('role') == 'assistant')
    & (pl.col('rank') != None)
    & (pl.col('rank') < 2)
).select(['message_id', 'parent_id', 'text', 'role', 'rank'])

ap_df = df.filter(
    (pl.col('role') == 'prompter')
    # & (pl.col('parent_id') != None)
).select(['message_id', 'parent_id', 'text', 'role'])


In [168]:
m_chains = []

def build_m_chains(p_row, m_chain=''):
    # if len(m_chain) < 1:
        # m_chain += (f"<|INPUT|>{p_row['text']}<|END_INPUT|>")
    if p_row['role'] == 'prompter':
        resp = r_df.filter(pl.col('parent_id') == p_row['message_id']).to_dicts()
        # print(f'resp len: {len(resp)} \nprompt resp:\n {resp}\n\n*******')
        if resp:
            m_chain += (f"<|INPUT|>{p_row['text']}<|END_INPUT|>")
            for resp in resp:
                _m_chain = (f"{m_chain}<|RESPONSE|>{resp['text']}<|END_RESPONSE|>")
                build_m_chains(resp, _m_chain)
    elif p_row['role'] == 'assistant':
        resp = ap_df.filter(pl.col('parent_id') == p_row['message_id']).to_dicts()
        # print(f'prompt: {resp}\n\n*******')
        if resp:
            for resp in resp:
                # _m_chain = (f"{m_chain}<|INPUT|>{p_row['text']}<|END_INPUT|>")
                build_m_chains(resp, m_chain)
    if not resp:
        m_chains.append(m_chain)
        # print(m_chain)
        # return m_chain
    
# m_chains = []

for row in p_df.iter_rows(named=True):
    build_m_chains(row)
    # print(m)
    # m_chains.append(m)
    # children = rg_df.filter(pl.col('parent_id') == row['message_id'])
    # print(row)
    # ld = {
    #     'prompt': row['text'],
    #     'responses': row['message_id'],
    # }



In [171]:
odf = pl.DataFrame({
    "message": m_chains
}).with_columns(
    pl.lit('oasst1').alias('source'),
    pl.lit('').alias('context'),
    pl.lit('').alias('system'),
    pl.lit('').alias('input'),
    pl.lit('').alias('response'),
    tags = ['instruct', 'reviewed', 'dialogue']
).select(['input', 'response', 'source', 'context', 'system', 'message', 'tags'])

with open(out_file, mode="ab") as f:
   odf.write_ndjson(f)